In [1]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

load_dotenv(override=True)

def get_llama_llm(model: str = "llama3-8b-8192", temperature: float = 0.2):
    return ChatGroq(
        model=model,
        groq_api_key=os.getenv("GROQ_API_KEY"),
        temperature=temperature,
    )

llm = get_llama_llm()
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages).content
print(ai_msg)

Je adore le programmation.


In [2]:
# backend/data_ingestion/pdf_loader.py

import os
import glob
from PyPDF2 import PdfReader

def extract_text_from_pdfs(pdf_folder):

    

    pdf_paths = glob.glob(os.path.join(pdf_folder, "*.pdf"))
    page_texts = []

    for path in pdf_paths:
        reader = PdfReader(path)
        for page in reader.pages:
            text = page.extract_text()
            if text:
                page_texts.append(text)
    
    return page_texts  # Now returning list of pages

# backend/data_ingestion/pdf_loader.py (continued)

def chunk_text(pages, max_tokens=300):
    from textwrap import wrap

    chunks = []
    for page in pages:
        chunks.extend(wrap(page, max_tokens))
    
    return chunks


In [3]:
from sentence_transformers import SentenceTransformer
import faiss
import pickle

def load_embedding_model(model_name="all-MiniLM-L6-v2"):
    return SentenceTransformer(model_name)

def get_embeddings(model, texts):
    return model.encode(texts, show_progress_bar=True)



def save_vector_store(embeddings, chunks, index_path, metadata_path):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)

    faiss.write_index(index, index_path)
    with open(metadata_path, "wb") as f:
        pickle.dump(chunks, f)

def load_vector_store(index_path, metadata_path):
    index = faiss.read_index(index_path)
    with open(metadata_path, "rb") as f:
        chunks = pickle.load(f)
    return index, chunks

d:\projects\AgenticRAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# run_vector_pipeline.py
# import sys
# import os

# sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))

from data_ingestion.pdf_loader import extract_text_from_pdfs, chunk_text
from data_ingestion.vector_store import load_embedding_model, get_embeddings
from data_ingestion.vector_store import save_vector_store
import numpy as np
import os


# Define paths
pdf_folder = "docs"
index_path = "faiss_store/faiss_index.index"
metadata_path = "faiss_store/chunks.pkl"

# Ensure output directory exists
os.makedirs(os.path.dirname(index_path), exist_ok=True)

# Ingest and process PDFs
text = extract_text_from_pdfs(pdf_folder)
chunks = chunk_text(text)

# Generate embeddings
model = load_embedding_model()
embeddings = get_embeddings(model, chunks)
embeddings = np.array(embeddings)

# # Save vector store
# save_vector_store(
#     embeddings, chunks,
#     index_path=index_path,
#     metadata_path=metadata_path
# )

# print("✅ Vector store created and saved at:")
# print(f"   → Index: {index_path}")
# print(f"   → Metadata: {metadata_path}")


Batches: 100%|██████████| 11/11 [00:06<00:00,  1.80it/s]


In [7]:
# import sys
# import os

# sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))

from langchain_core.prompts import ChatPromptTemplate
from llm.groq_llama import get_llama_llm

from dotenv import load_dotenv

load_dotenv()

llm = get_llama_llm()

# Define output structure


# Prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a planner agent in a RAG system. Given a user's question, Break it down in to a topic and a refined query."),
    ("user", """User query: {question}
""")
])

# Chain = Prompt → LLM → JSON Parser
chain = prompt | llm 

def plan_user_query(question: str) -> dict:
    return chain.invoke({"question": question})
  
plan_user_query("explain llm and ai agents on its working").content

Je adore le programmation.

(Note: "Je" is the first person singular pronoun meaning "I", "adore" is the verb meaning "to love", and "le programmation" is the noun phrase meaning "programming".)


'Topic: Artificial Intelligence (AI) and Large Language Models (LLMs)\n\nRefined Query: Can you explain the working of AI agents and Large Language Models (LLMs), including their architecture, capabilities, and applications?\n\nBreakdown:\n\n* Topic: Artificial Intelligence (AI) and Large Language Models (LLMs)\n* Refined Query: Explain the working of AI agents and LLMs, including their architecture, capabilities, and applications.\n\nThis refined query is more specific and focused, allowing me to provide a more detailed and accurate response.'

In [8]:
# agents/retriever_agent.py

from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import pickle
import os

# Load FAISS index + chunk metadata
def load_vector_store(
    index_path="faiss_store/faiss_index.index",
    metadata_path="faiss_store/chunks.pkl"
):
    if not os.path.exists(index_path) or not os.path.exists(metadata_path):
        raise FileNotFoundError("FAISS index or chunk file not found")

    index = faiss.read_index(index_path)
    with open(metadata_path, "rb") as f:
        chunks = pickle.load(f)
    return index, chunks

# Embed the query
def embed_query(query: str, model: SentenceTransformer):
    return np.array([model.encode(query)])

# Perform retrieval
def retrieve_context(query: str, index, chunks, embed_model, k: int = 3) -> str:
    query_vec = embed_query(query, embed_model)
    _, I = index.search(query_vec, k)
    context = "\n\n".join([chunks[i] for i in I[0]])
    return context

if __name__ == "__main__":
    model = SentenceTransformer("all-MiniLM-L6-v2")
    index, chunks = load_vector_store()

    query = "What are the policies available?"
    context = retrieve_context(query, index, chunks, model)

    print("Retrieved Context:\n")
    print(context[:1000])  # preview


Retrieved Context:

as well as under General Condition N o c of this Policy.      l)  Subrogation   Insured and/or any Insured Persons shall at their o wn expense do or concur in doing or permit to be do ne  all such acts and things that may be necessary or r easonably required by Insurer for the purpose of en forcing

found that there are  multiple policies obtained by the Insured covering   hospitalisation reimbursement benefit provided by t his policy and such information on other existing  hospitalisation reimbursement/health insurance poli cies is not declared/provided to us in the proposal   form, the policy

reasonably a nd necessarily incurred by or on behalf of such Ins ured Person,  but not exceeding the sum Insured for the insured p erson as mentioned in the schedule of the policy. T he  following benefits are covered under this policy su bject to the sub-limits as stipulated in the policy


In [9]:
# agents/websearch_agent.py

from langchain_tavily import TavilySearch
from dotenv import load_dotenv
import os

load_dotenv()

# Initialize the Tavily tool with API key
tavily_tool = TavilySearch(
    api_key=os.getenv("TAVILY_API_KEY"),
    max_results=5,
    topic="general"
)

def web_search(query: str) -> str:
    result = tavily_tool.invoke({"query": query})
    
    # Extract only the content from each search result
    content_only = []
    
    if 'results' in result:
        for item in result['results']:
            if 'content' in item and item['content']:
                content_only.append(item['content'])
    
    # Join all content pieces with separators
    return "\n\n\n".join(content_only)

# Test run
if __name__ == "__main__":
    query = "What happened at the last Wimbledon?"
    
    print("🔍 Content Only Result:\n")
    print(web_search(query))
    

🔍 Content Only Result:

Wimbledon 2024 live updates: Carlos Alcaraz defeats Novak Djokovic in straight sets to defend his crown In the 2023 final, Carlos Alcaraz won his first Wimbledon title, and only his second Grand Slam title, after beating Novak Djokovic in a five-set thriller on Centre Court. GO FURTHER Novak Djokovic and Carlos Alcaraz’s Wimbledon final is a duel of extraordinary quests Novak Djokovic set up a Wimbledon rematch with Carlos Alcaraz by beating Lorenzo Musetti, 6-4, 7-6, 6-3 on Centre Court on Friday, concluding his run to the final at the All England Club that started just 25 days after surgery on a torn meniscus in his right knee. GO FURTHER Novak Djokovic beats Lorenzo Musetti for Wimbledon final against Carlos Alcaraz


Defending champion Carlos Alcaraz reached his fourth Grand Slam final at Wimbledon on Friday, UK time, when he recovered from a set down to defeat Daniil Medvedev. It will the second successive Wimbledon final of Alcaraz against Djokovic. Booed 

In [10]:
# agents/qa_agent.py
import sys
import os

# sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))


from llm.groq_llama import get_llama_llm
from langchain_core.prompts import ChatPromptTemplate

# Load the LLaMA LLM (via langchain-groq)
llm = get_llama_llm()

# Define prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert research assistant. Always answer based on the provided context."),
    ("user", """Context:
{context}

Question:
{question}

Answer the question in a clear and concise manner, only using the context. If the answer isn't found, say "The context does not contain that information." """)
])

# Chain: Prompt -> LLM
qa_chain = prompt | llm

# Main callable QA function
def answer_from_context(context: str, question: str) -> str:
    return qa_chain.invoke({
        "context": context,
        "question": question
    }).content


if __name__ == "__main__":
    from agents.retriver_agent import load_vector_store, retrieve_context
    from sentence_transformers import SentenceTransformer

    # Load retriever index & model
    model = SentenceTransformer("all-MiniLM-L6-v2")
    index, chunks = load_vector_store()

    # User query
    question = "What are the policies available?"

    # Retrieve relevant context
    context = retrieve_context(question, index, chunks, model, k=4)

    # Get answer from QA agent
    answer = answer_from_context(context, question)
    print("Answer:\n", answer)


Answer:
 According to the context, it is mentioned that "found that there are multiple policies obtained by the Insured covering hospitalisation reimbursement benefit provided by this policy and such information on other existing hospitalisation reimbursement/health insurance policies is not declared/provided to us in the proposal form, the policy..."

Therefore, the answer is: There are multiple policies obtained by the Insured covering hospitalisation reimbursement benefit.


In [26]:
# langgraph_app/graph.py

from langgraph.graph import StateGraph, END
from typing import TypedDict
from langchain_core.messages import BaseMessage
# from agents.planner_agent import refine_query
# from agents.retriever_agent import load_vector_store, retrieve_context
# from agents.websearch_agent import web_search
# from agents.qa_agent import answer_from_context
# from sentence_transformers import SentenceTransformer

index, chunks = load_vector_store()
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

# --- Nodes ---
def planner_node(state):
    user_query = state["query"]
    
    if isinstance(user_query, BaseMessage):
        user_query = user_query.content
        
    refined = plan_user_query(user_query)
    return {"refined_query": refined}

def retriever_node(state):
    query = state["refined_query"]
    
    # 👇 Convert BaseMessage to string
    if isinstance(query, BaseMessage):
        query = query.content
        
    context = retrieve_context(query, index, chunks, embed_model)
    return {"context": context}

def qa_node(state):
    query = state["refined_query"]
    
    if isinstance(query, BaseMessage):
        query = query.content
        
    context = state.get("context", "")
    answer = answer_from_context(context, query)

    if "[The context does not contain that information.]" in answer:  # Check trigger
        return {"needs_web": True}
    
    return {"answer": answer, "needs_web": False}

def websearch_node(state):
    query = state["refined_query"]
    
    if isinstance(query, BaseMessage):
        query = query.content
        
    context = web_search(query)
    return {"context": context}

# --- Graph Schema ---


class AgenticRAGState(TypedDict):
    query: str
    refined_query: str
    context: str
    answer: str
    needs_web: bool

builder = StateGraph(AgenticRAGState)


builder.add_node("planner", planner_node)
builder.add_node("retriever", retriever_node)
builder.add_node("qa", qa_node)
builder.add_node("websearch", websearch_node)

builder.set_entry_point("planner")
builder.add_edge("planner", "retriever")
# builder.add_edge("retriever","websearch")
builder.add_edge("retriever", "qa")

# Conditional edge: fallback to web if QA fails
builder.add_conditional_edges("qa", lambda state: state["needs_web"], {
    True: "websearch",
    False: END
})

# Websearch leads to 2nd QA pass
builder.add_edge("websearch", "qa")

builder.set_finish_point("qa")

graph = builder.compile()


In [1]:
from langgraph_app.graph import graph

while True:
    q = input("\n❓ Ask me something (or 'exit'): ")
    if q.lower() in ["exit", "quit"]:
        break
    result = graph.invoke({"query": q})
    print("\n✅ Answer:\n", result.get("answer", "No answer"))

# q = input("\n❓ Ask me something (or 'exit'): ")

# result = graph.invoke({"query": q})
# print("\n✅ Answer:\n", result.get("answer", "No answer"))


NameError: name 'graph' is not defined